# Lab 2: K-Nearest Neighbours


## Version history

| Date | Author | Description |
|:----:|:------:|:------------|
2021-01-18 | Josiah Wang | First version |
2021-10-18 | Josiah Wang | Fixed typo in Distance Weighted K-NN Classifier evaluation. The last piece of code should say WeightedKNNClassifier(k) and not KNNClassifier(k), obviously! |

## Introduction

The aim of this lab exercise is to give you some practical experience to build a K-Nearest Neighbours (K-NN) classifier.

By the end of this lab exercise, you will have constructed different variants of a K-NN classifier as discussed in the lectures.

## The Iris dataset

Continuing where we left off in Lab 1, we will again work with the Iris dataset.

I will just copy and rerun my solutions from Lab 1. Feel free to copy your own implementations over.

In [3]:
import os
import numpy as np
from numpy.random import default_rng

# Download iris data if it does not exist
if not os.path.exists("iris.data"):
    !wget -O iris.data https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data


def read_dataset(filepath):
    """ Read in the dataset from the specified filepath

    Args:
        filepath (str): The filepath to the dataset file

    Returns:
        tuple: returns a tuple of (x, y, classes), each being a numpy array.
               - x is a numpy array with shape (N, K),
                   where N is the number of instances
                   K is the number of features/attributes
               - y is a numpy array with shape (N, ), and should be integers from 0 to C-1
                   where C is the number of classes
               - classes : a numpy array with shape (C, ), which contains the
                   unique class labels corresponding to the integers in y
    """

    x = []
    y_labels = []
    for line in open(filepath):
        if line.strip() != "": # handle empty rows in file
            row = line.strip().split(",")
            x.append(list(map(float, row[:-1])))
            y_labels.append(row[-1])

    [classes, y] = np.unique(y_labels, return_inverse=True)

    x = np.array(x)
    y = np.array(y)
    return (x, y, classes)


def split_dataset(x, y, test_proportion, random_generator=default_rng()):
    """ Split dataset into training and test sets, according to the given
        test set proportion.

    Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)
        y (np.ndarray): Class labels, numpy array with shape (N,)
        test_proprotion (float): the desired proportion of test examples
                                 (0.0-1.0)
        random_generator (np.random.Generator): A random generator

    Returns:
        tuple: returns a tuple of (x_train, x_test, y_train, y_test)
               - x_train (np.ndarray): Training instances shape (N_train, K)
               - x_test (np.ndarray): Test instances shape (N_test, K)
               - y_train (np.ndarray): Training labels, shape (N_train, )
               - y_test (np.ndarray): Test labels, shape (N_train, )
    """

    shuffled_indices = random_generator.permutation(len(x))
    n_test = round(len(x) * test_proportion)
    n_train = len(x) - n_test
    x_train = x[shuffled_indices[:n_train]]
    y_train = y[shuffled_indices[:n_train]]
    x_test = x[shuffled_indices[n_train:]]
    y_test = y[shuffled_indices[n_train:]]
    return (x_train, x_test, y_train, y_test)



(x, y, classes) = read_dataset("iris.data")

seed = 60012
rg = default_rng(seed)
x_train, x_test, y_train, y_test = split_dataset(x, y,
                                                 test_proportion=0.2,
                                                 random_generator=rg)
print(x_train.shape)
print(x_test.shape)


(120, 4)
(30, 4)


## K-NN Classifier

In Lab 1, you have constructed a (one) Nearest Neighbour classifier.

We will now try to generalise the nearest neighbour classifier as a **K-Nearest Neighours (K-NN)** classifier. For each test example, the classifier will predict the majority class label among the $K$ nearest training examples, again according to the Euclidean distance metric $d(x^{(i)}, x^{(q)})=\sqrt{\sum_f^F (x_f^{(i)} - x_f^{(q)})^2}$. If there is draw, choose one of the majority class labels arbitrarily, or at random.

Complete the `predict()` method of the `KNNClassifier` class. Note that the class now takes an optional hyperparameter `k` in its constructor.


In [55]:
class KNNClassifier:
    def __init__(self, k=5):
        """ K-NN Classifier.

        Args:
        k (int): Number of nearest neighbours. Defaults to 5.
        """
        self.k = k
        self.x_train = np.array([])
        self.y_train = np.array([])

    def fit(self, x, y):
        """ Fit the training data to the classifier.

        Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)
        y (np.ndarray): Class labels, numpy array with shape (N,)
        """
        self.x_train = x
        self.y_train = y

    def predict(self, x_test):
        """ Perform prediction given some examples.

        Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)

        Returns:
        y (np.ndarray): Predicted class labels, numpy array with shape (N,)
        """
        
        # TODO: Implement a K-NN classifier
        results = []
        for test_sample in x_test:
            distances = []
            #for each test sample, we find the k closest neighbours
            for train_index, train_sample in enumerate(self.x_train):
                
                sum = 0
                for feature_index, feature in enumerate(train_sample):
                    sum += (feature - test_sample[feature_index]) ** 2
                dist = np.sqrt(sum)
                distances.append([dist, train_index])

            
            #got the distances
            distances = sorted(distances, key=lambda x: x[0])
            # distances now contains the list [ [min, index], [2nd smallest, index], ... [largest, index] ]
            # need to find the k smallest indexes and find the average of their labels

            potential_labels = []
            for i in range(self.k):
                potential_labels.append(self.y_train[distances[i][1]])
            
            


            sorted_labels = sorted(potential_labels)
            #results.append(  sorted_labels[ int(np.floor(len(sorted_labels)/2)) ]  )
            results.append( max(set(sorted_labels), key = sorted_labels.count) )
            #print("of {a} labels {b} was chosen".format(a=sorted_labels, b=max(set(sorted_labels), key = sorted_labels.count)))
            #return 0


        return results

In [56]:
nn_classifier = KNNClassifier(k=5)
nn_classifier.fit(x_train, y_train)
nn_predictions = nn_classifier.predict(x_test)
print(nn_predictions)

of [1, 1, 1, 1, 1] labels 1 was chosen
of [2, 2, 2, 2, 2] labels 2 was chosen
of [1, 1, 1, 1, 1] labels 1 was chosen
of [2, 2, 2, 2, 2] labels 2 was chosen
of [0, 0, 0, 0, 0] labels 0 was chosen
of [1, 1, 1, 1, 1] labels 1 was chosen
of [2, 2, 2, 2, 2] labels 2 was chosen
of [0, 0, 0, 0, 0] labels 0 was chosen
of [0, 0, 0, 0, 0] labels 0 was chosen
of [2, 2, 2, 2, 2] labels 2 was chosen
of [2, 2, 2, 2, 2] labels 2 was chosen
of [2, 2, 2, 2, 2] labels 2 was chosen
of [1, 2, 2, 2, 2] labels 2 was chosen
of [0, 0, 0, 0, 0] labels 0 was chosen
of [2, 2, 2, 2, 2] labels 2 was chosen
of [0, 0, 0, 0, 0] labels 0 was chosen
of [0, 0, 0, 0, 0] labels 0 was chosen
of [2, 2, 2, 2, 2] labels 2 was chosen
of [1, 2, 2, 2, 2] labels 2 was chosen
of [1, 1, 1, 2, 2] labels 1 was chosen
of [2, 2, 2, 2, 2] labels 2 was chosen
of [1, 1, 1, 1, 1] labels 1 was chosen
of [1, 1, 1, 1, 2] labels 1 was chosen
of [1, 1, 1, 1, 1] labels 1 was chosen
of [1, 1, 1, 1, 1] labels 1 was chosen
of [0, 0, 0, 0, 0] labels

### Evaluation

Now, let's evaluate the accuracy of our K-NN classifier. We will test K from 1 to 20. Again, you should be able to achieve very high accuracy.

In [53]:
def compute_accuracy(y_gold, y_prediction):
    """ Compute the accuracy given the ground truth and predictions

    Args:
    y_gold (np.ndarray): the correct ground truth/gold standard labels
    y_prediction (np.ndarray): the predicted labels

    Returns:
    float : the accuracy
    """

    assert len(y_gold) == len(y_prediction)

    try:
        return np.sum(y_gold == y_prediction) / len(y_gold)
    except ZeroDivisionError:
        return 0

In [54]:
for k in range(1, 21):
    nn_classifier = KNNClassifier(k)
    nn_classifier.fit(x_train, y_train)
    nn_predictions = nn_classifier.predict(x_test)
    accuracy = compute_accuracy(y_test, nn_predictions)
    print(f"K={k}: {accuracy}")

K=1: 0.9666666666666667
K=2: 0.9666666666666667
K=3: 1.0
K=4: 1.0
K=5: 1.0
K=6: 1.0
K=7: 0.9666666666666667
K=8: 1.0
K=9: 0.9666666666666667
K=10: 1.0
K=11: 1.0
K=12: 0.9666666666666667
K=13: 0.9666666666666667
K=14: 0.9666666666666667
K=15: 0.9666666666666667
K=16: 0.9666666666666667
K=17: 0.9666666666666667
K=18: 0.9666666666666667
K=19: 0.9666666666666667
K=20: 0.9666666666666667


## Distance Weighted K-NN Classifier

Now, let us implement the **distance weighted** K-NN classifier as discussed in the lectures.
Pleae complete the `predict()` method for the `WeightedKNNClassifier` class below.

For each test example $x^{(q)}$, you will need to compute the weight $w^{(i)}_{q}$ for each training example $x^{(i)}$.

Then for each class label, sum the weights of the $K$ nearest examples. Assign the test example to the class label with largest sum.

The weight can be anything reasonable. For this tutorial, we will use the weight $w_{q}^{(i)}=\frac{1}{d(x^{(i)}, x^{(q)})}$, where $d(x^{(i)}, x^{(q)})=\sqrt{\sum_f^F (x_f^{(i)} - x_f^{(q)})^2}$ is the Euclidean distance.


In [124]:
class WeightedKNNClassifier:
    def __init__(self, k=5):
        """ K-NN Classifier.

        Args:
        k (int): Number of nearest neighbours. Defaults to 5.
        """
        self.k = k
        self.x_train = np.array([])
        self.y_train = np.array([])

    def fit(self, x, y):
        """ Fit the training data to the classifier.

        Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)
        y (np.ndarray): Class labels, numpy array with shape (N,)
        """
        self.x_train = x
        self.y_train = y

    def predict(self, x_test):
        """ Perform prediction given some examples.

        Args:
        x (np.ndarray): Instances, numpy array with shape (N,K)

        Returns:
        y (np.ndarray): Predicted class labels, numpy array with shape (N,)
        """
        # TODO: Implement a distance weighted K-NN classifier
        results = []
        for test_sample in x_test:
            distances = []
            #for each test sample, we find the k closest neighbours
            for train_index, train_sample in enumerate(self.x_train):
                
                sum = 0
                for feature_index, feature in enumerate(train_sample):
                    sum += (feature - test_sample[feature_index]) ** 2
               
                dist = np.sqrt(sum)
                weight = 1/dist
                distances.append([dist, train_index, weight])

            
            #got the distances
            distances = sorted(distances, key=lambda x: x[0])
            # distances now contains the list [ [min, index], [2nd smallest, index], ... [largest, index] ]
            # need to find the k smallest indexes and find the average of their labels

            potential_labels = []
            for i in range(self.k):
                potential_labels.append( [ self.y_train[distances[i][1]], distances[i][2] ] )
            
            
            weights = {}
            #print(potential_labels)

            for entry in potential_labels:
                #print(entry)
                #print("checking if {a} is in {b}".format( a = entry[0], b = list(weights.keys()) ))
                if entry[0] in (weights.keys()):
                    #print("is in")
                    #print("adding {a} to {b}".format(a = entry[1], b = weights[entry[0]]))
                    weights[entry[0]] += entry[1]
                else:
                    #print("not ere")
                    weights[entry[0]] = entry[1]


           # print("weights: {}".format(weights))

            flipped = {v: k for k, v in weights.items()}
            #print(flipped)
            min = np.inf
            for weight in flipped.keys():
                if weight <= min:
                    min = weight

            results.append(flipped[min])
            
        return results

        

        



In [125]:
nn_classifier = WeightedKNNClassifier(k=5)
nn_classifier.fit(x_train, y_train)
nn_predictions = nn_classifier.predict(x_test)
print(nn_predictions)

0.9799999999999999
1.02
0.059999999999999706
2.0099999999999985
11.090000000000002
2.3799999999999986
2.2199999999999998
0.629999999999999
0.9499999999999995
10.440000000000001
0.889999999999999
2.459999999999998
10.14
13.63
10.329999999999998
12.530000000000001
11.01
15.290000000000001
13.090000000000002
0.5699999999999995
8.430000000000001
3.0
2.359999999999999
12.940000000000003
17.360000000000003
14.480000000000002
0.539999999999999
12.710000000000003
12.93
7.78
1.1000000000000005
1.4699999999999984
12.510000000000002
2.99
0.8999999999999991
0.38999999999999957
4.42
13.090000000000002
3.9499999999999984
1.0099999999999998
2.63
0.2899999999999997
2.159999999999999
0.6600000000000001
1.779999999999999
3.1199999999999997
1.4099999999999993
1.369999999999999
1.14
0.8499999999999988
0.1899999999999997
1.1899999999999986
12.51
0.020000000000000122
0.9499999999999995
12.860000000000001
4.269999999999999
0.06999999999999981
0.2399999999999997
13.33
14.370000000000003
1.8699999999999999
11.

C:\Users\omar_\AppData\Local\Temp\ipykernel_12128\3733711257.py:43: RuntimeWarning: divide by zero encountered in double_scalars
  weight = 1/dist


### Evaluation

Again, we will evaluate the accuracy of the distance weighted K-NN classifier for K=1 to 20. As this is a small and relatively easy dataset, you should get similar results as with the simple K-NN classifier.

You might observe the benefits of the distance weighted K-NN classifier better when training on larger and noisier datasets.

In [121]:
for k in range(1, 21):
    nn_classifier = WeightedKNNClassifier(k)
    nn_classifier.fit(x_train, y_train)
    nn_predictions = nn_classifier.predict(x_test)
    accuracy = compute_accuracy(y_test, nn_predictions)
    print(f"K={k}: {accuracy}")

C:\Users\omar_\AppData\Local\Temp\ipykernel_12128\4280476782.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  weight = 1/dist


K=1: 0.9666666666666667
K=2: 1.0
K=3: 0.9333333333333333
K=4: 0.9
K=5: 0.8666666666666667
K=6: 0.8333333333333334
K=7: 0.8666666666666667
K=8: 0.8333333333333334
K=9: 0.8
K=10: 0.7333333333333333
K=11: 0.7
K=12: 0.7
K=13: 0.7333333333333333
K=14: 0.7333333333333333
K=15: 0.7333333333333333
K=16: 0.6666666666666666
K=17: 0.6333333333333333
K=18: 0.6333333333333333
K=19: 0.6333333333333333
K=20: 0.5666666666666667


## Summary

You have built a K-Nearest Neighbour classifier (and the distance-weighted variant)! Because the Iris dataset is a small and simple dataset, you may only observe a small improvement in accuracy (if any) compared to a simple one-nearest neighbour classifier.

And that is it for this week's lab tutorials. We will not have a decision tree tutorial for this course. The coursework itself will contain a short guide to help you think about implementing your own decision trees.

In the next lab tutorial, you will be implementing different evaluation metrics and performing cross-validation.
